# Adder
Quantum computers have the function of being able to perform the same calculations as conventional computers. Check the circuit for adder.

## What we'll learn this time
1. Implementing Binary Adder Using Quantum Gates
2. Using quantum superposition, we can realize multiple adder in a single circuit

## Install Blueqat
Install Blueqat from pip.

In [1]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## Addition of binary numbers
Adder uses a CCX gate to perform digit up and a CX gate to perform addition. In this case, we will do a quantum circuit of binary addition, a+b = cd. This time, we will implement four types of addition according to the values of a and b. Each addition is,

0+0 = 00 => 0000  
0+1 = 01 => 0101  
1+0 = 01 => 1001  
1+1 = 10 => 1110  

The first two qubits are the input values a and b, and the second two qubits are the output values c and d. I've built a separate circuit for the input of a and b and a circuit for the addition, and used it several times. For data input, use the X gate to flip 0 to 1.

The circuit part of the additon looks like this. * is a controlled bit.

```
a ---*---*------- a
b ---*---|---*--- b
0 ---X---|---|--- c
0 -------X---X--- d
```

In [2]:
#import tools
from blueqat import Circuit

#additon part
adder = Circuit().ccx[0,1,2].cx[0,3].cx[1,3]

In [3]:
#0+0
(Circuit() + adder).m[:].run(shots=100)

Counter({'0000': 100})

In [4]:
#0+1
(Circuit().x[1] + adder).m[:].run(shots=100)

Counter({'0101': 100})

In [5]:
#1+0
(Circuit().x[0] + adder).m[:].run(shots=100)

Counter({'1001': 100})

In [6]:
#1+1
(Circuit().x[0,1] + adder).m[:].run(shots=100)

Counter({'1110': 100})

As you can see, we were able to add up.

## Addition using superposition
Now, instead of filling in the data one by one with the X-gate, let's use the H-gate to do the addition.

In [7]:
#Use H gate as input instead of X gate
(Circuit().h[0,1] + adder).m[:].run(shots=100)

Counter({'1110': 27, '0101': 18, '0000': 34, '1001': 21})

Using the Hadamal Gate, I got 4 answers, about 1/4 each. If you create a universal addition circuit like this, you can perform calculations using the superposition state.

## Addition using entanglement
Next, instead of the H-gate, we try to add a+b=1 using quantum entanglement.

In [8]:
#Make an entangled state of 01 and 10
(Circuit().h[0].cx[0,1].x[0] + adder).m[:].run(shots=100)

Counter({'0101': 50, '1001': 50})

Since the input values 01 and 10 are entangled, the addition of these two will come out about half by half.

--------

## Advanced: General addition
It implements the addition of common decimal numbers to each other.  
Let's consider the sum of $a, b$.
$a, b$ is $a = a_n ... a_0$, $b = b_n ... b_0$ and can be expressed in binary numbers.  
(where n is based on the larger number.)

The circuit is as following.

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F489c8ea6-130c-d44b-4bae-4d86f64556c2.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=e67fbf71779fa7ebb443e93663c56dc4">

$c_i$ is a carry bit.  
This circuit is divided into two parts: the carry part and the sum part.

we consider these two parts first.

### Carry up

<img src="../tutorial-ja/img/100_0.png" width="30%">

Let $c_i, a_i, b_i, c_{i+1}$ from the top, and $c_{i+1}$ shows the carry-up part.

### Sum

<img src="../tutorial-ja/img/100_1.png" width="30%">

Let $c_i, a_i, b_i$ from the top and the sum of the three numbers will appear in $b_i$.

### implementation

General addition is:

1. Using the carry circuit calculates the carry bit.  
2. Using CX gate undo the last part of the carry.  
3. Using the sum circuit outputs the sum of each position to $b_n$.  
4. Using the reverse carry circuit undo the value of that level.  
5. Using the sum circuit outputs the sum of each position to $b_i$.  
6. repetition of 4,5.

$a+b$ is output in $b_{n+1} ... b_0$.  
First of all, we make the carry and its reverse circuit and the sum circuit.

In [1]:
from blueqat import Circuit

def carry(i):
    return Circuit().ccx[i+1,i+2,i+3].cx[i+1,i+2].ccx[i,i+2,i+3]

def carry_reverse(i):
    return Circuit().ccx[i,i+2,i+3].cx[i+1,i+2].ccx[i+1,i+2,i+3]

def sum(i):
    return Circuit().cx[i+1,i+2].cx[i,i+2]

We'll also create a function to turn a decimal number into a binary number.

In [2]:
def tobinary(A):
    return bin(A)[2:]

tobinary(10)

'1010'

Next, create a function that maps a binary number to a circuit.

In [3]:
def digits(a,b): 
     # change to the binary
     aa = tobinary(a)  
     bb = tobinary(b)  
     alen = len(aa)  
     blen = len(bb)  

     # get the value n
     maxlen = max(alen,blen) 
     if alen>blen: 
         bb = bb.zfill(alen) 
     elif blen>alen: 
         aa = aa.zfill(blen) 

     # mapping
     str = '' 
     for i in range(maxlen): 
         str += '0' + aa[maxlen-i-1] + bb[maxlen-i-1] 
     str += '0' 
     return str

digits(2,2)

'0000110'

The X gate needs to be applied to match the mapped values, because the initial state of the circuit is all zero.

In [4]:
def toX(a): 
     cir = Circuit(len(a)) 
     for i in range(len(a)): 
         if a[i] == "1": 
             cir += Circuit().x[i] 
     return cir

toX("101").m[:].run(shots=100)

Counter({'101': 100})

Finally, consider the output part.  
The output is a binary number, so it is converted to a decimal number.

In [5]:
def todecimal(A):
    return int(str(A),2) 

todecimal(1001)

9

The circuit outputs a mixture of $a_i, b_i$ and $c_i$.  
We get only $b_i$.

In [6]:
def getb(result): 
     str = result[-1] 
     digi = int((len(result)-1)/3) 
     for i in range(digi): 
         str += result[-2-i*3] 
     return todecimal(str)

getb("0000110")

2

### General circuit

In [7]:
def plus(a,b): 
     # Mapping a binary number to a circuit
     qubits = len(digits(a,b)) 
     cir1 = toX(digits(a,b)) 
     digi = int((len(digits(a,b))-1)/3) 

     # first carry circuit
     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry(i*3) 

     # last digit
     cir3 = Circuit(qubits).cx[-3,-2] + sum((digi-1)*3) 

     # Output the sum to bi
     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += carry_reverse((digi-i-2)*3) 
         cir4 += sum((digi-i-2)*3)

     result = (cir1 + cir2 + cir3 + cir4).m[:].run(shots=1) 
     return getb(result.most_common()[0][0])

Let's actually calculate it.

In [8]:
plus(2,2)

4

In [9]:
plus(13,15)

28

In [10]:
plus(70,90)

160

The last calculation takes some time, but we were able to implement a general adder.

### References
V. Vedral, A. Barenco, A. Ekert, "Quantum Networks for Elementary Arithmetic Operations",  
(Submitted on 16 Nov 1995)  
https://arxiv.org/pdf/quant-ph/9511018.pdf

# Subtractor
Quantum computers have the function of being able to perform the same calculations as conventional computers. Check the circuit for Subtractor.

## What we'll learn this time
1. Implementing Binary Subtractor Using Quantum Gates
2. Using quantum superposition, we can realize multiple Substractor in a single circuit

## Subtraction of binary numbers
Subtractor uses a CCX gate to perform minus sign or a plus sign and a CX gate to perform Subtraction. In this case, we will do a quantum circuit of binary Subtraction, a-b = cd. This time, we will implement four types of Subtraction according to the values of a and b. Each Subtraction is,

0-0 = 00 => 0000  
0-1 = 11 => 0111  
1-0 = 01 => 1001  
1-1 = 00 => 1100  

The first two qubits are the input values a and b, and the second two qubits are the output values c and d. I've built a separate circuit for the input of a and b and a circuit for the addition, and used it several times. For data input, use the X gate to flip 0 to 1.

The circuit part of the additon looks like this. * is a controlled bit.

```
a ---X---*---X---*------- a
b -------*-------|---*--- b
0 -------X-------|---|--- c
0 ---------------X---X--- d
```

In [1]:
#import tools
from blueqat import Circuit

#subtraction part
subtractor = Circuit().x[0].ccx[0,1,2].x[0].cx[0,3].cx[1,3]

In [2]:
#0-0
(Circuit() + subtractor).m[:].run(shots=100)

Counter({'0000': 100})

In [3]:
#0-1
(Circuit().x[1] + subtractor).m[:].run(shots=100)

Counter({'0111': 100})

In [4]:
#1-0
(Circuit().x[0] + subtractor).m[:].run(shots=100)

Counter({'1001': 100})

In [5]:
#1-1
(Circuit().x[0,1] + subtractor).m[:].run(shots=100)

Counter({'1100': 100})

As you can see, we were able to calculate.

## subtraction using superposition
Now, instead of filling in the data one by one with the X-gate, let's use the H-gate to do the subtraction.

In [7]:
#Use H gate as input instead of X gate
(Circuit().h[0,1] + subtractor).m[:].run(shots=100)

Counter({'1001': 31, '1100': 17, '0111': 23, '0000': 29})

Using the Hadamal Gate, I got 4 answers, about 1/4 each. If you create a universal subtraction circuit like this, you can perform calculations using the superposition state.

## subtraction using entanglement
Next, instead of the H-gate, we try to subtract a-b=0 using quantum entanglement.

In [8]:
#Make an entangled state of 00 and 11
(Circuit().h[0].cx[0,1] + subtractor).m[:].run(shots=100)

Counter({'0000': 42, '1100': 58})

Since the input values 00 and 11 are entangled, the addition of these two will come out about half by half.

--------

## Advanced: General subtraction
It implements the subtraction of common decimal numbers to each other.  
The subtraction circuit can be implemented by reversing the addition circuit.

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F489c8ea6-130c-d44b-4bae-4d86f64556c2.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=e67fbf71779fa7ebb443e93663c56dc4">

Consider the circuit from the right. Input $a, a+b$ to output $b$.  
It can be displayed in binary numbers such as $a = a_n ... a_0$, $b = b_n ... b_0$.  
(where n is based on the larger number.)

$c_i$ is a carry down.  

### Carry down

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F00361854-e782-0da0-3386-f59f668a1ada.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=58d1d41b0893b16bfb24bfd546328ff2" width="60%">

The first part of the circuit combines these to obtain the carry down.

### Overflow bit
The discrimination is given by $b_{n+1}$. This is called the overflow bit.  

When $a < a+b$, $b_{n+1}$ = 0  
When $a > a+b$, $b_{n+1}$ = 1.

If $a > a+b$, $2^{n+1} - b$ will be output.

### Implementation

General subtraction is:

1. Using the reverse and sum circuits calculates the carry bit.  
2. Check the overflow and store it in $b_{n+1}$.  
3. Using the carry circuit calculate the defference.

$b$ is output in $b_n ... b_0$, and $b_{n+1}$ is the overflow bit.  
We'll prepare it for implementation. This is the same as the addition circuit.

In [9]:
from blueqat import Circuit

def carry(i):
    return Circuit().ccx[i+1,i+2,i+3].cx[i+1,i+2].ccx[i,i+2,i+3]

def carry_reverse(i):
    return Circuit().ccx[i,i+2,i+3].cx[i+1,i+2].ccx[i+1,i+2,i+3]

def sum_reverse(a):
    return Circuit().cx[a,a+2].cx[a+1,a+2]

def tobinary(A):
    return bin(A)[2:]

def digits(a,b): 
     # change to the binary
     aa = tobinary(a)  
     bb = tobinary(b)  
     alen = len(aa)  
     blen = len(bb)  

     # get the value n
     maxlen = max(alen,blen) 
     if alen>blen: 
         bb = bb.zfill(alen) 
     elif blen>alen: 
         aa = aa.zfill(blen) 

     # mapping
     str = '' 
     for i in range(maxlen): 
         str += '0' + aa[maxlen-i-1] + bb[maxlen-i-1] 
     str += '0' 
     return str

def toX(a): 
     cir = Circuit(len(a)) 
     for i in range(len(a)): 
         if a[i] == "1": 
             cir += Circuit().x[i] 
     return cir

def todecimal(A):
    return int(str(A),2) 

def getb(result): 
     str = result[-1] 
     digi = int((len(result)-1)/3) 
     for i in range(digi): 
         str += result[-2-i*3] 
     return todecimal(str)

In [10]:
def minus(a,ab): 
     # swap a for ab
     c = ab
     ab = a
     a = c

     # Mapping a binary number to a circuit
     qubits = len(digits(a,ab)) 
     cir1 = toX(digits(a,ab)) 
     digi = int((len(digits(a,ab))-1)/3) 

     # first carry circuit and sum reverse circuit
     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += sum_reverse(i*3)
         cir4 += carry(i*3) 

     # last digit
     cir3 = sum_reverse((digi-1)*3) + Circuit(qubits).cx[-3,-2] 

     # Output the subtraction to bi
     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry_reverse((digi-1-i)*3)

     result = (cir1 + cir4 + cir3 + cir2).m[:].run(shots=1) 
     return getb(result.most_common()[0][0])

Let's actually calculate it.

In [11]:
minus(8,2)

6

In [12]:
minus(4,2)

2

In [13]:
minus(50,24)

26

We've calculated it. Incidentally, for $a > a + b$

In [14]:
minus(2,4)

14

This is also calculated.

### References
V. Vedral, A. Barenco, A. Ekert, "Quantum Networks for Elementary Arithmetic Operations",  
(Submitted on 16 Nov 1995)  
https://arxiv.org/pdf/quant-ph/9511018.pdf

# Multiplier
Here we see at the basic multiplier circuit with adder circuit.

## About multiplier of binary number
Basic multiplier circuit is consist of CCX for multiply 2 binary number.

$$0*0=0\\
0*1=0\\
1*0=0\\
1*1=1$$

and then sum up each rank


## Example
Let's solve 

$$1*2 = ?$$

We express this calculation on binary number as 

$$01*10 = 0010$$


```
    01
*  10
-------
    00
 01
-------
    0
 0
-------
 0010
  ```
  
Now we pepare a0,a2 as a.b0,b2 as b.
c0,c21,c22,c4 as c the process after multiply a and b.
z4,z8 as carry.
and x0 to x8 as the result.

<img src="https://github.com/mdrft/Blueqat_tutorials_ja/blob/master/docs/img/010_basic_multi01.png?raw=1">


In [15]:
from blueqat import Circuit

C1 = Circuit().ccx[0,1,2].ccx[1,3,5].ccx[0,4,6].ccx[3,4,7].ccx[5,6,8].ccx[7,8,9].cx[2,10].cx[5,11].cx[6,11].cx[7,12].cx[8,12].cx[9,13] 

#00 * 00 = 0000
(Circuit() + C1).m[:].run(shots=100)

Counter({'00000000000000': 100})

In [16]:
#01 * 01 = 0001
(Circuit().x[0].x[1] + C1).m[:].run(shots=100)

Counter({'11100000001000': 100})

In [17]:
#10 * 01 = 0010
(Circuit().x[3].x[1] + C1).m[:].run(shots=100)

Counter({'01010100000100': 100})

In [18]:
#11 * 10 = 0110
(Circuit().x[0].x[3].x[4] + C1).m[:].run(shots=100)

Counter({'10011011000110': 100})

In [19]:
#10 * 11 = 0110
(Circuit().x[1].x[3].x[4] + C1).m[:].run(shots=100)

Counter({'01011101000110': 100})

In [20]:
#11 * 11 = 1001
(Circuit().x[0].x[1].x[3].x[4] + C1).m[:].run(shots=100) 

Counter({'11111111111001': 100})

Now we see the result of multiplier. The array of last 4qubits shows the result.

# Modulus
This time we'll find the modulus by using a general addition and subtraction circuit.

The circuit is:

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2Ff161e722-f727-b6fb-d02b-24e40b5b63b1.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=da298eace648814b5ccfe4affa9301ca">

$a,b$ is $0 < a,b < N$. The last bit is called the temporary bit. It checks the overflow.

## Implementation procedure
In order to find $a+b\mod N$, we need to compare $a+b$ and $N$.  

When $a+b>N$,  
$a+b-N = (a+b) \mod N$

When $a+b<N$, 
$a+b = (a+b) \mod N$

We will implement it in a quantum circuit.

When $a+b>N$,

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F5c97b6fd-425b-c42b-057a-68d9cb47e457.jpeg?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=3711f239a543dc4aa175f4636adde888">

When $a+b<N$,

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2Fcb7bc1bf-e713-f7d4-9cdc-4bc47bb6eba8.jpeg?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=425f2f86ef9d5b5c27b1c1f1ddfab936">

It uses the surplus qubits and the value of the topmost qubit of the adder to make the case.  
If $a+b<N$, no extra bits are used and no extra operations are required.

## Example
Let's look at a simple example.

When $3 + 5 < 7$,  
$(3 + 5) \mod 7 = 1$

When $3 + 5 > 11$,    
$(3 + 5) \mod 11 = 8$

This is what we are trying to implement by using quantum circuits.

### Implementation
We'll prepare it for implementation. This is the same as the addition circuit and the subtraction circuit.

we will add another N to the circuit above to make the temporary bit and N correspond to each other.   
The initial state is as follows.

```
c0 --
a0 --
b0 --
c1 --
..
n0 --
n1 --
..
t  --
n0 --
n1 --
..
```

Also, each digit is $N$.

In [21]:
from blueqat import Circuit

def carry(a):
    return Circuit().ccx[a+1,a+2,a+3].cx[a+1,a+2].ccx[a,a+2,a+3]

def carry_reverse(a):
    return Circuit().ccx[a,a+2,a+3].cx[a+1,a+2].ccx[a+1,a+2,a+3]

def sum(a):
    return Circuit().cx[a+1,a+2].cx[a,a+2] 

def sum_reverse(a):
    return Circuit().cx[a,a+2].cx[a+1,a+2]

def tobinary(A):
    return bin(A)[2:] 

# get the digit number
def digits2(a,b,n): 
     aa = tobinary(a)  
     bb = tobinary(b)  
     nn = tobinary(n)  

     nlen = len(nn)  
     aa = aa.zfill(nlen) 
     bb = bb.zfill(nlen) 

     str = '' 
     for i in range(nlen): 
         str += '0' + aa[nlen-i-1] + bb[nlen-i-1] 
     str += '0' 

     for i in range(nlen): 
         str += nn[nlen-i-1]  

     str += '0'

     for i in range(nlen): 
        str += nn[nlen-i-1]

     return str

# initial states
def toX(a): 
     cir = Circuit(len(a)) 
     for i in range(len(a)): 
         if a[i] == "1": 
             cir += Circuit().x[i] 
     return cir

# adder
def plus(a,b,n): 
     qubits = len(digits2(a,b,n))
     digi = len(tobinary(n))

     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry(i*3) 

     cir3 = Circuit(qubits).cx[(digi-1)*3+1,(digi-1)*3+2] + sum((digi-1)*3)

     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += carry_reverse((digi-i-2)*3) 
         cir4 += sum((digi-i-2)*3)

     cir_plus = cir2 + cir3 + cir4
     return cir_plus

# adder_inverse
def minus(a,ab,n): 
     qubits = len(digits2(a,ab,n))
     digi = len(tobinary(n))

     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += sum_reverse(i*3)
         cir4 += carry(i*3) 

     cir3 = sum_reverse((digi-1)*3) + Circuit(qubits).cx[(digi-1)*3+1,(digi-1)*3+2] 

     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry_reverse((digi-1-i)*3) 

     cir_minus = cir4 + cir3 + cir2
     return cir_minus

# swap a and N
def swap(n):
    digi = len(tobinary(n))
    cir = Circuit(5*digi+2)
    for i in range(digi):
        cir += Circuit(5*digi+2).swap[3*i+1,3*digi+1+i]
    return cir

# to decimal number
def todecimal(A):
    return int(str(A),2) 

# get b from the circuit
def getb(result,n): 
     str = ""
     digi = len(tobinary(n)) 
     for i in range(digi): 
         str += result[3*(digi-i)-1]
     return todecimal(str)

### General circuit

In [22]:
def adder_mod(a,b,n):
    digi = len(tobinary(n))

    # first part
    part1 = toX(digits2(a,b,n)) + plus(a,b,n) + swap(n) + minus(a,b,n)

    # set the overflow to a temporary bit
    part2 = Circuit(5*digi+2).x[digi*3].cx[digi*3,digi*4+1].x[digi*3]

    # returns N from a temporary bit.
    part3 = Circuit(5*digi+2)
    for i in range(digi):
        part3 += Circuit(5*digi+2).ccx[4*digi+2+i,4*digi+1,3*i+1]

    # last part
    part4 = minus(a,b,n)+Circuit(5*digi+2).cx[digi*3,digi*4+1]+plus(a,b,n)
    
    result = (part1+part2+part3+plus(a,b,n)+part3+swap(n)+part4).m[:].run(shots=1)
    return getb(result.most_common()[0][0],n)

Let's actually calculate it.

In [23]:
adder_mod(4,3,5)

2

In [24]:
adder_mod(4,4,5)

3

In [25]:
adder_mod(1,5,6)

0

We've calculated it.